# Projet d'import des données Historique Zendesk vers BigQuery

## Introduction
1. Imports des bibliothèques nécessaires
2. Définition des variables et des configurations
3. Manipulation des données JSON
4. Création des fonctions pour le chargement des données
5. Exécution du processus de chargement

## 1. Importation des bibliothèques

In [16]:
print("... Début du chargement des bibliothèques ...")
import json                         # Manipulation JSON -> Déjà prévu dans Python
import pandas as pd                 # Manipulation données -> pip install pandas

#instalation Gcoud CLI + créer des id locaux pour mon compte Google
# Documentations :
# - https://cloud.google.com/bigquery/docs/authentication?hl=fr
# - https://cloud.google.com/sdk/docs/install?hl=fr
# - https://cloud.google.com/bigquery/docs/authentication/getting-started?hl=fr

from google.cloud import bigquery   # Liaison BigQuery -> pip install google-cloud-bigquery
from google.oauth2 import service_account
from google.cloud.exceptions import NotFound
import os
print("... Bibliothèques chargées ...")

... Début du chargement des bibliothèques ...
... Bibliothèques chargées ...


## 2. Variables et configuration

#### Fichiers JSON

In [14]:
exportZendesk1 = 'file1.json'   # Export
exportZendesk2 = 'file2.json'   # Données en base

#### Variables BigQuery

In [20]:


#project_id = "your_projectID"                          # Identifiant du projet BigQuery
#dataset_id = "your_datasetID"                  # Identifiant du dataset BigQuery
#table_name = "your_tablename"                                   # Nom de la table de destination
#json_file_path = "your_path"                     # Chemin d'accès à l'export JSON

## 3. Manipulation données JSON

#### Merge des 2 fichiers

In [ ]:
print("... Ouverture du premier fichier en cours ...")
with open(exportZendesk1) as f:  
    data1 = json.load(f)
print("... Premier fichier ouvert ...")  

print("... Ouverture du deuxième fichier en cours ...")  
with open(exportZendesk2) as f:  
    data2 = json.load(f)
print("... Deuxième fichier ouvert ...")  

print("... Fusion des 2 fichiers ...")
data1.update(data2)
print("... Fusion terminé ...")

print("... écriture du nouveau fichier ...")
with open('merged.json', 'w') as f:  
    json.dump(data1, f)
print("... écriture du nouveau fichier terminé ...")

#### Vérification et suppression des doublons

## 4. Fonctions (chargement et insertion des données)

In [23]:
# Initialisation du client BigQuery avec une clé de compte de service
print("... Initialisation du client BigQuery ...")
#client = bigquery.Client(credentials=credentials, project=project_id)
#credentials = service_account.Credentials.from_service_account_file(credentials_path)
client = bigquery.Client(project=project_id)
print("... Initialisation terminé")

# Référence au dataset et à la table
dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_name)

# Vérification de l'existence du dataset
try:
    client.get_dataset(dataset_ref)
    print(f"Dataset {dataset_id} found.")
except NotFound:
    print(f"Dataset {dataset_id} not found.")
    exit(1)

# Vérification de l'existence de la table
try:
    client.get_table(table_ref)
    print(f"Table {table_name} found.")
except NotFound:
    print(f"Table {table_name} not found. Creating table schema.")

# Configuration du job de chargement
print("... Confirguration ...")
job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND
print("... Configuration terminé ...")

# Chargement des données JSON dans la table BigQuery
print("... Début du chargement des données ...")
with open(json_file_path, "rb") as source_file:                                           # rb = read binary => meilleur compatibilité et intégrité des données
    load_job = client.load_table_from_file(source_file, table_ref, job_config=job_config)
load_job.result()                                                                           # Attendre la fin du job de chargement
print("... Chargement des données terminé ...")

print(f"Les données du fichier {json_file_path} ont été chargées avec succès dans {table_name}")

... Initialisation du client BigQuery ...
... Initialisation terminé
Dataset syncari found.
Table zd_tickets found.
... Confirguration ...
... Configuration terminé ...
... Début du chargement des données ...


Forbidden: 403 POST https://bigquery.googleapis.com/upload/bigquery/v2/projects/lumapps-internal-bi/jobs?uploadType=resumable: Access Denied: Dataset lumapps-internal-bi:syncari: Permission bigquery.tables.create denied on dataset lumapps-internal-bi:syncari (or it may not exist).